In [19]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [8]:
!python ../scripts/mimiccxr/export_cxrlt2024_training_labels_for_fact_classification.py \
--fact_encoder_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_encoder_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--positive_fraction 0.04

2024-07-20 00:06:42,171 - INFO - Loading CXR-LT 2024 training data
2024-07-20 00:06:43,514 - INFO - train_labels.shape: (258871, 40)
2024-07-20 00:06:43,514 - INFO - Computing class embeddings
checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']
  0%|                                                    | 0/40 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=441,505879158912416539).pkl
len(self.cache["hashes"]) = 599219
self.cache["embeddings"].shape = (599219, 128)
100%|███████████████████████████████████████████| 40/40 [00:00<00:00, 89.03it/s]
2024-07-20 00:06:44,194 - INFO - class_embeddings.shape: (40, 128)
2024-07-20 00:06:44,194 - INFO - Splitting the training data into train and validation sets
2024-07-20 00:06:45,147 - INFO - len(train_indices): 234750
2024-07-20 00:06:45,147 - INFO - len(val_indices): 24121
2

In [13]:
from medvqa.utils.files import load_pickle

tmp = load_pickle('/mnt/workspace/pamessina/medvqa-workspace/cache/mimiccxr/cxrlt2024_official_training_labels_for_fact_classification(n_train=240529,n_val=18342,pf=0.03).pkl')
# tmp = load_pickle('/mnt/workspace/pamessina/medvqa-workspace/cache/mimiccxr/cxrlt2024_official_training_labels_for_fact_classification(n_train=234750,n_val=24121,pf=0.04).pkl')

In [16]:
tmp.keys()

dict_keys(['dicom_ids', 'labels', 'train_indices', 'val_indices', 'class_names', 'class_embeddings'])

In [17]:
tmp['labels'][tmp['val_indices']].sum(0)

array([ 354, 6830,   22,  429, 7330,   27, 1762, 4195,  382, 2716,  137,
        347, 1306,  240,  379,   88,   81,  907,   76,   21,  250, 7782,
        607,  816, 1262, 7425,   58,  344,   77, 4129,   52, 1527,  143,
         91,  969,   27,  307, 8612,  305,  160])

In [24]:
!python ../train_phrase_grounding.py \
--epochs 200 \
--batches_per_epoch 600 \
--max_images_per_batch 45 \
--max_phrases_per_batch 800 \
--max_phrases_per_image 50 \
--val_batch_size_factor 3.0 \
--raw_image_encoding "densenet-121" \
--img_aug_mode "random-color-and-spatial" \
--image_size 416 416 \
--image_local_feat_size 1024 \
--num_regions 169 \
--regions_width 13 \
--regions_height 13 \
--phrase_embedding_size 128 \
--phrase_grounding_mode "film_layers_plus_sigmoid_attention_and_custom_classifier" \
--visual_feature_proj_size 256 \
--visual_grounding_hidden_size 256 \
--phrase_mlp_hidden_dims 256 128 \
--num_train_workers 2 \
--num_val_workers 2 \
--gradient_accumulation_steps 10 \
--optimizer_name "adamw" \
--scheduler "exp-warmup+decay+cyclicdecay" \
--lr 1e-6 \
--warmup_decay_and_cyclic_decay_args "1e-6,3,8e-5,8,1e-6,8e-5,8,1e-6" \
--mimiccxr_exclude_noisy_images \
--binary_multilabel_classif_loss_name "focal+bce+wbce" \
--use_attention_regularization_loss \
--phrase_classifier_loss_weight 2.0 \
--use_cxrlt2024_challenge_split \
--use_cxrlt2024_official_labels \
--cxrlt2024_official_training_labels_for_fact_classification_filepath \
"/mnt/workspace/pamessina/medvqa-workspace/cache/mimiccxr/cxrlt2024_official_training_labels_for_fact_classification(n_train=240529,n_val=18342,pf=0.03).pkl" \
--save

script's arguments:
   epochs: 200
   batches_per_epoch: 600
   max_images_per_batch: 45
   max_phrases_per_batch: 800
   max_phrases_per_image: 50
   val_batch_size_factor: 3.0
   checkpoint_folder: None
   pretrained_checkpoint_folder_path: None
   pretrained_checkpoint_folder_paths: None
   freeze_image_encoder: False
   raw_image_encoding: densenet-121
   num_regions: 169
   image_local_feat_size: 1024
   image_encoder_pretrained_weights_path: None
   image_encoder_dropout_p: 0
   yolov8_model_name_or_path: None
   yolov8_model_alias: None
   phrase_embedding_size: 128
   regions_width: 13
   regions_height: 13
   qkv_size: None
   phrase_grounding_mode: film_layers_plus_sigmoid_attention_and_custom_classifier
   phrase_classifier_hidden_size: None
   transf_d_model: None
   transf_nhead: None
   transf_dim_feedforward: None
   transf_dropout: 0
   transf_num_layers: None
   visual_feature_proj_size: 256
   visual_grounding_hidden_size: 256
   phrase_mlp_hidden_dims: [256, 128]
   

loss 4.00792, cxrlt2024o_att_reg_loss 0.49185, cxrlt2024o_phrcls_loss 3.51607, cxrlt2024o_prc_auc 0.05894, 225.99 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.48941, cxrlt2024o_phrcls_loss 1.85283, cxrlt2024o_prc_auc 0.08229, 94.58 secs
New checkpoint saved: checkpoint_1_cxrlt2024o_prc_auc=0.0823.pt
---- Epoch 2/200
(1) Training stage (lr = 0.000004) ...
loss 3.85586, cxrlt2024o_att_reg_loss 0.48047, cxrlt2024o_phrcls_loss 3.37540, cxrlt2024o_prc_auc 0.05904, 231.57 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.46657, cxrlt2024o_phrcls_loss 1.74478, cxrlt2024o_prc_auc 0.08752, 94.48 secs
New checkpoint saved: checkpoint_2_cxrlt2024o_prc_auc=0.0875.pt
---- Epoch 3/200
(1) Training stage (lr = 0.000019) ...
loss 2.82435, cxrlt2024o_att_reg_loss 0.39631, cxrlt2024o_phrcls_loss 2.42804, cxrlt2024o_prc_auc 0.06039, 233.92 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.29050, cxrlt2024o_phrcls_loss 1.45651, cxrlt2024o_prc_auc 0.09317, 95.23 secs
New checkpoint s

cxrlt2024o_att_reg_loss 0.10954, cxrlt2024o_phrcls_loss 1.40682, cxrlt2024o_prc_auc 0.12900, 101.39 secs
New checkpoint saved: checkpoint_22_cxrlt2024o_prc_auc=0.1290.pt
---- Epoch 23/200
(1) Training stage (lr = 0.000023) ...
loss 1.83521, cxrlt2024o_att_reg_loss 0.14763, cxrlt2024o_phrcls_loss 1.68758, cxrlt2024o_prc_auc 0.10697, 233.92 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.10535, cxrlt2024o_phrcls_loss 1.39560, cxrlt2024o_prc_auc 0.12993, 102.13 secs
New checkpoint saved: checkpoint_23_cxrlt2024o_prc_auc=0.1299.pt
---- Epoch 24/200
(1) Training stage (lr = 0.000012) ...
loss 1.83926, cxrlt2024o_att_reg_loss 0.14754, cxrlt2024o_phrcls_loss 1.69172, cxrlt2024o_prc_auc 0.10885, 234.78 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.10615, cxrlt2024o_phrcls_loss 1.38612, cxrlt2024o_prc_auc 0.13015, 96.76 secs
New checkpoint saved: checkpoint_24_cxrlt2024o_prc_auc=0.1302.pt
---- Epoch 25/200
(1) Training stage (lr = 0.000007) ...
loss 1.80804, cxrlt2024o_att_reg

loss 1.78732, cxrlt2024o_att_reg_loss 0.14535, cxrlt2024o_phrcls_loss 1.64197, cxrlt2024o_prc_auc 0.11204, 236.12 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.11041, cxrlt2024o_phrcls_loss 1.34738, cxrlt2024o_prc_auc 0.13894, 107.27 secs
New checkpoint saved: checkpoint_45_cxrlt2024o_prc_auc=0.1389.pt
---- Epoch 46/200
(1) Training stage (lr = 0.000043) ...
loss 1.79187, cxrlt2024o_att_reg_loss 0.14507, cxrlt2024o_phrcls_loss 1.64680, cxrlt2024o_prc_auc 0.11373, 234.61 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.10155, cxrlt2024o_phrcls_loss 1.37020, cxrlt2024o_prc_auc 0.14155, 105.94 secs
New checkpoint saved: checkpoint_46_cxrlt2024o_prc_auc=0.1416.pt
---- Epoch 47/200
(1) Training stage (lr = 0.000023) ...
loss 1.77700, cxrlt2024o_att_reg_loss 0.14289, cxrlt2024o_phrcls_loss 1.63411, cxrlt2024o_prc_auc 0.11825, 234.83 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.10143, cxrlt2024o_phrcls_loss 1.37395, cxrlt2024o_prc_auc 0.14295, 103.52 secs
New check

loss 1.73826, cxrlt2024o_att_reg_loss 0.14064, cxrlt2024o_phrcls_loss 1.59762, cxrlt2024o_prc_auc 0.13027, 232.73 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.10074, cxrlt2024o_phrcls_loss 1.34892, cxrlt2024o_prc_auc 0.15363, 103.89 secs
New checkpoint saved: checkpoint_67_cxrlt2024o_prc_auc=0.1536.pt
---- Epoch 68/200
(1) Training stage (lr = 0.000001) ...
loss 1.76419, cxrlt2024o_att_reg_loss 0.14206, cxrlt2024o_phrcls_loss 1.62213, cxrlt2024o_prc_auc 0.12943, 234.62 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.10067, cxrlt2024o_phrcls_loss 1.35141, cxrlt2024o_prc_auc 0.15354, 103.67 secs
---- Epoch 69/200
(1) Training stage (lr = 0.000080) ...
loss 1.75473, cxrlt2024o_att_reg_loss 0.14458, cxrlt2024o_phrcls_loss 1.61015, cxrlt2024o_prc_auc 0.12466, 233.76 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.10205, cxrlt2024o_phrcls_loss 1.36193, cxrlt2024o_prc_auc 0.15198, 100.42 secs
---- Epoch 70/200
(1) Training stage (lr = 0.000043) ...
loss 1.77986, cxr

loss 1.70309, cxrlt2024o_att_reg_loss 0.14037, cxrlt2024o_phrcls_loss 1.56272, cxrlt2024o_prc_auc 0.14552, 232.97 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.10014, cxrlt2024o_phrcls_loss 1.30655, cxrlt2024o_prc_auc 0.16968, 96.77 secs
---- Epoch 90/200
(1) Training stage (lr = 0.000003) ...
loss 1.73840, cxrlt2024o_att_reg_loss 0.14114, cxrlt2024o_phrcls_loss 1.59727, cxrlt2024o_prc_auc 0.14363, 228.02 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.10056, cxrlt2024o_phrcls_loss 1.31703, cxrlt2024o_prc_auc 0.17017, 109.50 secs
New checkpoint saved: checkpoint_90_cxrlt2024o_prc_auc=0.1702.pt
---- Epoch 91/200
(1) Training stage (lr = 0.000002) ...
loss 1.72536, cxrlt2024o_att_reg_loss 0.14107, cxrlt2024o_phrcls_loss 1.58429, cxrlt2024o_prc_auc 0.14293, 232.52 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.10011, cxrlt2024o_phrcls_loss 1.31606, cxrlt2024o_prc_auc 0.17036, 104.01 secs
New checkpoint saved: checkpoint_91_cxrlt2024o_prc_auc=0.1704.pt
---- Epoch

New checkpoint saved: checkpoint_111_cxrlt2024o_prc_auc=0.1843.pt
---- Epoch 112/200
(1) Training stage (lr = 0.000012) ...
loss 1.68182, cxrlt2024o_att_reg_loss 0.13893, cxrlt2024o_phrcls_loss 1.54288, cxrlt2024o_prc_auc 0.16376, 231.44 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09836, cxrlt2024o_phrcls_loss 1.29670, cxrlt2024o_prc_auc 0.18470, 103.11 secs
New checkpoint saved: checkpoint_112_cxrlt2024o_prc_auc=0.1847.pt
---- Epoch 113/200
(1) Training stage (lr = 0.000007) ...
loss 1.67737, cxrlt2024o_att_reg_loss 0.13946, cxrlt2024o_phrcls_loss 1.53791, cxrlt2024o_prc_auc 0.16488, 229.34 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09802, cxrlt2024o_phrcls_loss 1.29391, cxrlt2024o_prc_auc 0.18437, 99.48 secs
---- Epoch 114/200
(1) Training stage (lr = 0.000003) ...
loss 1.68287, cxrlt2024o_att_reg_loss 0.13846, cxrlt2024o_phrcls_loss 1.54441, cxrlt2024o_prc_auc 0.16061, 231.93 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09821, cxrlt2024o_phrcls_lo

loss 1.68336, cxrlt2024o_att_reg_loss 0.13837, cxrlt2024o_phrcls_loss 1.54500, cxrlt2024o_prc_auc 0.15860, 233.86 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09781, cxrlt2024o_phrcls_loss 1.27148, cxrlt2024o_prc_auc 0.18865, 111.71 secs
---- Epoch 135/200
(1) Training stage (lr = 0.000023) ...
loss 1.69151, cxrlt2024o_att_reg_loss 0.13935, cxrlt2024o_phrcls_loss 1.55215, cxrlt2024o_prc_auc 0.15559, 233.68 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09750, cxrlt2024o_phrcls_loss 1.28278, cxrlt2024o_prc_auc 0.19106, 109.34 secs
---- Epoch 136/200
(1) Training stage (lr = 0.000012) ...
loss 1.67060, cxrlt2024o_att_reg_loss 0.13831, cxrlt2024o_phrcls_loss 1.53229, cxrlt2024o_prc_auc 0.16534, 233.13 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09715, cxrlt2024o_phrcls_loss 1.27455, cxrlt2024o_prc_auc 0.19216, 106.26 secs
New checkpoint saved: checkpoint_136_cxrlt2024o_prc_auc=0.1922.pt
---- Epoch 137/200
(1) Training stage (lr = 0.000007) ...
loss 1.65418,

loss 1.65267, cxrlt2024o_att_reg_loss 0.13599, cxrlt2024o_phrcls_loss 1.51668, cxrlt2024o_prc_auc 0.16768, 234.46 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09786, cxrlt2024o_phrcls_loss 1.26167, cxrlt2024o_prc_auc 0.19422, 99.58 secs
---- Epoch 158/200
(1) Training stage (lr = 0.000043) ...
loss 1.66121, cxrlt2024o_att_reg_loss 0.13656, cxrlt2024o_phrcls_loss 1.52465, cxrlt2024o_prc_auc 0.16407, 234.68 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09811, cxrlt2024o_phrcls_loss 1.27735, cxrlt2024o_prc_auc 0.19652, 102.81 secs
---- Epoch 159/200
(1) Training stage (lr = 0.000023) ...
loss 1.66925, cxrlt2024o_att_reg_loss 0.13675, cxrlt2024o_phrcls_loss 1.53249, cxrlt2024o_prc_auc 0.17466, 234.75 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09786, cxrlt2024o_phrcls_loss 1.27169, cxrlt2024o_prc_auc 0.19817, 103.65 secs
New checkpoint saved: checkpoint_159_cxrlt2024o_prc_auc=0.1982.pt
---- Epoch 160/200
(1) Training stage (lr = 0.000012) ...
loss 1.66851, 

cxrlt2024o_att_reg_loss 0.09643, cxrlt2024o_phrcls_loss 1.27179, cxrlt2024o_prc_auc 0.20339, 96.79 secs
---- Epoch 181/200
(1) Training stage (lr = 0.000080) ...
loss 1.63426, cxrlt2024o_att_reg_loss 0.13580, cxrlt2024o_phrcls_loss 1.49847, cxrlt2024o_prc_auc 0.18596, 238.46 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09688, cxrlt2024o_phrcls_loss 1.29979, cxrlt2024o_prc_auc 0.19339, 96.74 secs
---- Epoch 182/200
(1) Training stage (lr = 0.000043) ...
loss 1.67120, cxrlt2024o_att_reg_loss 0.13774, cxrlt2024o_phrcls_loss 1.53347, cxrlt2024o_prc_auc 0.17244, 238.19 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09894, cxrlt2024o_phrcls_loss 1.27042, cxrlt2024o_prc_auc 0.19945, 96.12 secs
---- Epoch 183/200
(1) Training stage (lr = 0.000023) ...
loss 1.66473, cxrlt2024o_att_reg_loss 0.13611, cxrlt2024o_phrcls_loss 1.52862, cxrlt2024o_prc_auc 0.17199, 237.88 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09683, cxrlt2024o_phrcls_loss 1.27197, cxrlt2024o_prc_au

In [ ]:
!python ../train_phrase_grounding.py \
--pretrained_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/phrase_grounding/20240720_003302_cxrlt2024(Off)_PhraseGrounder(dn121,FiLM_SigmoidAttention,128,256,256,256,1,256-128)" \
--epochs 200 \
--batches_per_epoch 650 \
--max_images_per_batch 45 \
--max_phrases_per_batch 800 \
--max_phrases_per_image 50 \
--val_batch_size_factor 3.0 \
--raw_image_encoding "densenet-121" \
--img_aug_mode "random-color-and-spatial" \
--image_size 416 416 \
--image_local_feat_size 1024 \
--num_regions 169 \
--regions_width 13 \
--regions_height 13 \
--phrase_embedding_size 128 \
--phrase_grounding_mode "film_layers_plus_sigmoid_attention_and_custom_classifier" \
--visual_feature_proj_size 256 \
--visual_grounding_hidden_size 256 \
--phrase_mlp_hidden_dims 256 128 \
--num_train_workers 2 \
--num_val_workers 2 \
--gradient_accumulation_steps 10 \
--optimizer_name "adamw" \
--scheduler "exp-warmup+decay+cyclicdecay" \
--lr 1e-6 \
--warmup_decay_and_cyclic_decay_args "1e-6,3,1e-4,8,1e-6,1e-4,8,1e-6" \
--mimiccxr_exclude_noisy_images \
--binary_multilabel_classif_loss_name "focal+bce+wbce" \
--phrase_classifier_loss_weight 2.0 \
--use_cxrlt2024_challenge_split \
--cxrlt2024_do_balanced_sampling \
--use_cxrlt2024_official_labels \
--cxrlt2024_official_training_labels_for_fact_classification_filepath \
"/mnt/workspace/pamessina/medvqa-workspace/cache/mimiccxr/cxrlt2024_official_training_labels_for_fact_classification(n_train=240529,n_val=18342,pf=0.03).pkl" \
--save

script's arguments:
   epochs: 200
   batches_per_epoch: 650
   max_images_per_batch: 45
   max_phrases_per_batch: 800
   max_phrases_per_image: 50
   val_batch_size_factor: 3.0
   checkpoint_folder: None
   pretrained_checkpoint_folder_path: /mnt/data/pamessina/workspaces/medvqa-workspace/models/phrase_grounding/20240720_003302_cxrlt2024(Off)_PhraseGrounder(dn121,FiLM_SigmoidAttention,128,256,256,256,1,256-128)
   pretrained_checkpoint_folder_paths: None
   freeze_image_encoder: False
   raw_image_encoding: densenet-121
   num_regions: 169
   image_local_feat_size: 1024
   image_encoder_pretrained_weights_path: None
   image_encoder_dropout_p: 0
   yolov8_model_name_or_path: None
   yolov8_model_alias: None
   phrase_embedding_size: 128
   regions_width: 13
   regions_height: 13
   qkv_size: None
   phrase_grounding_mode: film_layers_plus_sigmoid_attention_and_custom_classifier
   phrase_classifier_hidden_size: None
   transf_d_model: None
   transf_nhead: None
   transf_dim_feedforwa

Group sizes: [32948, 22986, 22569, 20510, 17768, 13915, 13795, 9545, 8812, 8338, 8004, 7462, 6396, 4514, 3453, 3355, 3167, 3083, 2769, 2601, 2542, 2453, 2433, 2235, 1906, 1853, 1842, 1469, 1315, 998, 790, 694, 637, 595, 553, 552, 463, 176, 144, 140, 108]
  len(indices) = 32948, weight = 3380.3375208043904
  len(indices) = 22986, weight = 3041.35685547107
  len(indices) = 22569, weight = 3024.7536909555574
  len(indices) = 20510, weight = 2938.9795463519754
  len(indices) = 17768, weight = 2813.3687596156055
  len(indices) = 13915, weight = 2607.758881815128
  len(indices) = 13795, weight = 2600.663258875812
  len(indices) = 9545, weight = 2310.715869437525
  len(indices) = 8812, weight = 2250.7968883149265
  len(indices) = 8338, weight = 2209.946575784679
  len(indices) = 8004, weight = 2180.0619800445174
  len(indices) = 7462, weight = 2129.4354369677103
  len(indices) = 6396, weight = 2020.90406028146
  len(indices) = 4514, weight = 1789.2726626049134
  len(indices) = 3453, weight = 

loss 2.18151, cxrlt2024o_phrcls_loss 2.18151, cxrlt2024o_prc_auc 0.23495, 250.76 secs
(2) Validation stage ...
cxrlt2024o_phrcls_loss 1.18432, cxrlt2024o_prc_auc 0.21105, 103.26 secs
---- Epoch 14/200
(1) Training stage (lr = 0.000052) ...
loss 2.17793, cxrlt2024o_phrcls_loss 2.17793, cxrlt2024o_prc_auc 0.23406, 251.43 secs
(2) Validation stage ...
cxrlt2024o_phrcls_loss 1.17564, cxrlt2024o_prc_auc 0.21365, 96.32 secs
---- Epoch 15/200
(1) Training stage (lr = 0.000027) ...
loss 2.16214, cxrlt2024o_phrcls_loss 2.16214, cxrlt2024o_prc_auc 0.25110, 250.28 secs
(2) Validation stage ...
cxrlt2024o_phrcls_loss 1.16577, cxrlt2024o_prc_auc 0.21608, 94.85 secs
New checkpoint saved: checkpoint_15_cxrlt2024o_prc_auc=0.2161.pt
---- Epoch 16/200
(1) Training stage (lr = 0.000014) ...
loss 2.15532, cxrlt2024o_phrcls_loss 2.15532, cxrlt2024o_prc_auc 0.25380, 252.39 secs
(2) Validation stage ...
cxrlt2024o_phrcls_loss 1.15331, cxrlt2024o_prc_auc 0.21690, 97.74 secs
New checkpoint saved: checkpoint_16

---- Epoch 41/200
(1) Training stage (lr = 0.000007) ...
loss 2.03674, cxrlt2024o_phrcls_loss 2.03674, cxrlt2024o_prc_auc 0.34034, 251.60 secs
(2) Validation stage ...
cxrlt2024o_phrcls_loss 1.14743, cxrlt2024o_prc_auc 0.23185, 102.32 secs
New checkpoint saved: checkpoint_41_cxrlt2024o_prc_auc=0.2319.pt
---- Epoch 42/200
(1) Training stage (lr = 0.000004) ...
loss 2.03640, cxrlt2024o_phrcls_loss 2.03640, cxrlt2024o_prc_auc 0.33850, 251.14 secs
(2) Validation stage ...
cxrlt2024o_phrcls_loss 1.15149, cxrlt2024o_prc_auc 0.23152, 101.02 secs
---- Epoch 43/200
(1) Training stage (lr = 0.000002) ...
loss 2.02927, cxrlt2024o_phrcls_loss 2.02927, cxrlt2024o_prc_auc 0.34122, 252.07 secs
(2) Validation stage ...
cxrlt2024o_phrcls_loss 1.15088, cxrlt2024o_prc_auc 0.23153, 97.31 secs
---- Epoch 44/200
(1) Training stage (lr = 0.000001) ...
loss 2.03180, cxrlt2024o_phrcls_loss 2.03180, cxrlt2024o_prc_auc 0.34041, 252.22 secs
(2) Validation stage ...
cxrlt2024o_phrcls_loss 1.15449, cxrlt2024o_prc_a

cxrlt2024o_phrcls_loss 1.15670, cxrlt2024o_prc_auc 0.23383, 101.21 secs
---- Epoch 73/200
(1) Training stage (lr = 0.000007) ...
loss 1.92812, cxrlt2024o_phrcls_loss 1.92812, cxrlt2024o_prc_auc 0.42070, 251.20 secs
(2) Validation stage ...
cxrlt2024o_phrcls_loss 1.16511, cxrlt2024o_prc_auc 0.23639, 98.52 secs
New checkpoint saved: checkpoint_73_cxrlt2024o_prc_auc=0.2364.pt
---- Epoch 74/200
(1) Training stage (lr = 0.000004) ...
loss 1.91989, cxrlt2024o_phrcls_loss 1.91989, cxrlt2024o_prc_auc 0.43353, 257.44 secs
(2) Validation stage ...
cxrlt2024o_phrcls_loss 1.16834, cxrlt2024o_prc_auc 0.23727, 95.38 secs
New checkpoint saved: checkpoint_74_cxrlt2024o_prc_auc=0.2373.pt
---- Epoch 75/200
(1) Training stage (lr = 0.000002) ...
loss 1.93001, cxrlt2024o_phrcls_loss 1.93001, cxrlt2024o_prc_auc 0.42052, 254.70 secs
(2) Validation stage ...
cxrlt2024o_phrcls_loss 1.16574, cxrlt2024o_prc_auc 0.23686, 95.00 secs
---- Epoch 76/200
(1) Training stage (lr = 0.000001) ...
loss 1.92777, cxrlt2024o

New checkpoint saved: checkpoint_104_cxrlt2024o_prc_auc=0.2381.pt
---- Epoch 105/200
(1) Training stage (lr = 0.000007) ...
loss 1.85273, cxrlt2024o_phrcls_loss 1.85273, cxrlt2024o_prc_auc 0.48196, 246.96 secs
(2) Validation stage ...
cxrlt2024o_phrcls_loss 1.19880, cxrlt2024o_prc_auc 0.23952, 93.39 secs
New checkpoint saved: checkpoint_105_cxrlt2024o_prc_auc=0.2395.pt
---- Epoch 106/200
(1) Training stage (lr = 0.000004) ...
loss 1.83423, cxrlt2024o_phrcls_loss 1.83423, cxrlt2024o_prc_auc 0.48496, 246.61 secs
(2) Validation stage ...
cxrlt2024o_phrcls_loss 1.19163, cxrlt2024o_prc_auc 0.23984, 93.92 secs
New checkpoint saved: checkpoint_106_cxrlt2024o_prc_auc=0.2398.pt
---- Epoch 107/200
(1) Training stage (lr = 0.000002) ...
loss 1.83953, cxrlt2024o_phrcls_loss 1.83953, cxrlt2024o_prc_auc 0.48195, 246.63 secs
(2) Validation stage ...
cxrlt2024o_phrcls_loss 1.19267, cxrlt2024o_prc_auc 0.23896, 93.53 secs
---- Epoch 108/200
(1) Training stage (lr = 0.000001) ...
loss 1.83916, cxrlt2024o